In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def simulate_and_plot(W0, Y0, X0, Z0, W_birth, Y_birth, W_death, Y_death,
                      X_size, Z_size, X_rate, Z_rate, W_rate, Y_rate,
                      Time, use_X, use_Z, show_phase_only, Y_size=1, W_size=1):
    """
    Simulates population dynamics for species W, Y, and, optionally, interactions with X and Z.
    """
    dt = 0.01
    t = np.arange(0, Time + dt, dt)
    W = np.zeros_like(t)
    Y = np.zeros_like(t)
    X = np.zeros_like(t)
    Z = np.zeros_like(t)
    W[0], Y[0], X[0], Z[0] = W0, Y0, X0, Z0

    # Placeholders for equilibrium lines; update as needed with actual calculations.
    W_equil1, W_equil2, Y_equil1, Y_equil2 = np.nan, np.nan, np.nan, np.nan

    for i in range(1, len(t)):
        # Base dynamics for W and Y
        dW = -W_birth * Y_size * (W[i - 1]**2) * Y[i - 1] + W_birth * Y_size * W[i - 1] * Y[i - 1] - W_death * W[i - 1]
        dY = -Y_birth * Y_size * (Y[i - 1]**2) * W[i - 1] + Y_birth * Y_size * W[i - 1] * Y[i - 1] - Y_death * Y[i - 1]
        
        # Modification by X and Z if enabled
        if use_X:
            dW += X_rate * X_size / W_size * (X[i - 1] * (1 - W[i - 1])) - W_rate * W[i - 1]
        if use_Z:
            dY += Z_rate * Z_size / Y_size * (Z[i - 1] * (1 - Y[i - 1])) - Y_rate * Y[i - 1]
        
        # Dynamics for X and Z (feedback from W and Y)
        dX = W_rate * W_size / X_size * (W[i - 1] * (1 - X[i - 1])) - X_rate * X[i - 1]
        dZ = Y_rate * Y_size / Z_size * (Y[i - 1] * (1 - Z[i - 1])) - Z_rate * Z[i - 1]

        # Update populations via Euler's method
        W[i] = W[i - 1] + dt * dW
        Y[i] = Y[i - 1] + dt * dY
        X[i] = X[i - 1] + dt * dX
        Z[i] = Z[i - 1] + dt * dZ

    # Plotting
    if not show_phase_only:
        plt.figure(figsize=(12, 5))
        # Plot X and Z if enabled
        if use_X:
            plt.plot(t, X, label=r'$X_t$', color='lightgreen')
        if use_Z:
            plt.plot(t, Z, label=r'$Z_t$', color='skyblue')
        # Plot W and Y trajectories
        plt.plot(t, W, label=r'$W_t$', color='darkgreen')
        plt.plot(t, Y, label=r'$Y_t$', color='darkblue')
        # Equilibrium lines (if computed)
        if not np.isnan(W_equil1):
            plt.axhline(W_equil1, color='darkgreen', linestyle='--', label=r'$W_{(eq)}^+$')
        if not np.isnan(W_equil2):
            plt.axhline(W_equil2, color='darkgreen', linestyle='--', label=r'$W_{(eq)}^-$')
        if not np.isnan(Y_equil1):
            plt.axhline(Y_equil1, color='darkblue', linestyle=':', label=r'$Y_{(eq)}^+$')
        if not np.isnan(Y_equil2):
            plt.axhline(Y_equil2, color='darkblue', linestyle=':', label=r'$Y_{(eq)}^-$')
        plt.xlabel('Time')
        plt.ylabel('Population')
        plt.title('Population Dynamics Over Time')
        plt.ylim(0, 1)
        plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
        plt.tight_layout()
        plt.show()
    else:
        # Phase plot setup
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        axs[0].plot(W, Y, label='W vs Y', color='purple')
        if use_X and use_Z:
            axs[0].plot(X, Z, label='X vs Z', color='brown', linestyle='--')
        axs[0].set_xlabel('W')
        axs[0].set_ylabel('Y')
        axs[0].set_title('Phase Plot: W vs Y')
        axs[0].set_xlim(0, 1)
        axs[0].set_ylim(0, 1)
        axs[0].grid(True)
        axs[0].legend(loc='best')
        plt.show()

# --- Slider Layout ---
slider_layout = widgets.Layout(width='250px')

# Column 1: Initial Values
W0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description="W0",
                                style={'handle_color': 'darkgreen'}, layout=slider_layout)
X0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.36, description="X0",
                                style={'handle_color': 'lightgreen'}, layout=slider_layout)
Y0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description="Y0",
                                style={'handle_color': 'darkblue'}, layout=slider_layout)
Z0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description="Z0",
                                style={'handle_color': 'lightskyblue'}, layout=slider_layout)

col1 = widgets.VBox([W0_slider, X0_slider, Y0_slider, Z0_slider])

# Column 2: Birth and Death Parameters
W_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.7, description="W_birth",
                                     style={'handle_color': 'darkgreen'}, layout=slider_layout)
W_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description="W_death",
                                     style={'handle_color': 'darkgreen'}, layout=slider_layout)
Y_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.9, description="Y_birth",
                                     style={'handle_color': 'darkblue'}, layout=slider_layout)
Y_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.15, description="Y_death",
                                     style={'handle_color': 'darkblue'}, layout=slider_layout)

col2 = widgets.VBox([W_birth_slider, W_death_slider, Y_birth_slider, Y_death_slider])

# Column 3: Parameters for X and Z
X_size_slider = widgets.FloatSlider(min=0.01, max=10.0, step=0.01, value=1, description="X_size",
                                    style={'handle_color': 'lightgreen'}, layout=slider_layout)
X_rate_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.1, description="X_rate",
                                    style={'handle_color': 'lightgreen'}, layout=slider_layout)
Z_size_slider = widgets.FloatSlider(min=0.01, max=10.0, step=0.01, value=1, description="Z_size",
                                    style={'handle_color': 'lightskyblue'}, layout=slider_layout)
Z_rate_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.05, description="Z_rate",
                                    style={'handle_color': 'lightskyblue'}, layout=slider_layout)

col3 = widgets.VBox([X_size_slider, X_rate_slider, Z_size_slider, Z_rate_slider])

# Column 4: Time, Toggles, and Additional Rates
Time_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="Time", layout=slider_layout)
phase_only_checkbox = widgets.Checkbox(value=False, description="Phase Plot Only", layout=slider_layout)
X_checkbox = widgets.Checkbox(value=True, description="Enable X", layout=slider_layout)
Z_checkbox = widgets.Checkbox(value=True, description="Enable Z", layout=slider_layout)
W_rate_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.02, description="W_rate",
                                    style={'handle_color': 'darkgreen'}, layout=slider_layout)
Y_rate_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.02, description="Y_rate",
                                    style={'handle_color': 'darkblue'}, layout=slider_layout)

col4 = widgets.VBox([Time_slider, phase_only_checkbox, X_checkbox, Z_checkbox, W_rate_slider, Y_rate_slider])

# Combine slider columns into a single control panel
controls = widgets.HBox([col1, col2, col3, col4])

# Bind the sliders to the simulation function using interactive_output
out = widgets.interactive_output(simulate_and_plot, {
    'W0': W0_slider, 'Y0': Y0_slider, 'X0': X0_slider, 'Z0': Z0_slider,
    'W_birth': W_birth_slider, 'Y_birth': Y_birth_slider,
    'W_death': W_death_slider, 'Y_death': Y_death_slider,
    'X_size': X_size_slider, 'Z_size': Z_size_slider,
    'X_rate': X_rate_slider, 'Z_rate': Z_rate_slider,
    'W_rate': W_rate_slider, 'Y_rate': Y_rate_slider,
    'Time': Time_slider,
    'use_X': X_checkbox, 'use_Z': Z_checkbox,
    'show_phase_only': phase_only_checkbox
})

# Display the interactive widgets and output
display(controls, out)

Output()